In [1]:
#Download ResNet18

import torch
import torchvision.models as models
model = torch.hub.load('pytorch/vision:v0.8.0', 'resnet18', pretrained=True)
model.load_state_dict(torch.load('resnet18_final.pt', map_location=torch.device('cpu')))
model.eval()

Using cache found in /Users/Ekaterina/.cache/torch/hub/pytorch_vision_v0.8.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Data processing functions

In [2]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import os

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

preprocess = transforms.Compose([
    transforms.Resize(256),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.1, 0.1, 0.1]),
])


class DataImg(Dataset):
    def __init__(self, path_1, path_2, path_3):
        self.path_1 = os.listdir(path_1)
        for i in range(len(self.path_1)):
            self.path_1[i] = os.path.join(path_1, self.path_1[i]) 
            
        self.path_2 = os.listdir(path_2)
        for i in range(len(self.path_2)):
            self.path_2[i] = os.path.join(path_2, self.path_2[i]) 
            
        self.path_3 = os.listdir(path_3)
        for i in range(len(self.path_3)):
            self.path_3[i] = os.path.join(path_3, self.path_3[i]) 
            
        
        self.labels = np.concatenate((np.zeros(len(self.path_1)), np.zeros(len(self.path_2)), np.ones(len(self.path_3))))  
        self.imgs = np.concatenate((self.path_1, self.path_2, self.path_3))

    def __getitem__(self, index):
        img = Image.open(self.imgs[index]) 
        label = self.labels[index]
        return preprocess(img), label

    def __len__(self):
        return len(self.imgs)

# Define dataloaders

In [3]:
batch_size = 124
device = 'cpu'
epochs = 80

In [4]:
import numpy as np

data = DataImg('dataset/collis', 'dataset/collis', 'dataset/right_position')
proportion = [0.8, 0.1, 0.1]
size_train = int(proportion[0] * data.__len__())
size_valid = int(proportion[1] * data.__len__())
size_test = data.__len__() - size_train - size_valid
trainset, validset, testset = torch.utils.data.random_split(data, [size_train, size_valid , size_test])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)
validloader = torch.utils.data.DataLoader(validset, batch_size=100, shuffle=False)

# Functions for training and testing

In [5]:
import numpy as np
import torch.optim as optim
from IPython.display import clear_output
import matplotlib.pyplot as plt
import scikitplot as skplt
import matplotlib.pyplot as plt

def test(model, testloader, device):
    model.to(device)
    acc = 0
    full_labels = []
    full_outputs = []
    for x, labels in testloader:
        x, labels = x.to(device), labels.to(device)
        outputs = model(x)
        
        full_labels += list(labels)
        full_outputs += list(outputs)
        
        pred = outputs.argmax(dim=1, keepdim=True) 
        acc += pred.eq(labels.view_as(pred)).sum().item()
    acc = acc / len(testloader.dataset)
    
    skplt.metrics.plot_roc_curve(full_labels, full_outputs)
    plt.savefig('graphs/auc.png')
    plt.show()
    
    return acc, full_labels, full_outputs

def train_and_validate(model, trainloader, validloader, testloader, epochs, device = 'cuda'):
    model.to(device)
    model.train()
    
    lr = 0.1
    optimizer = optim.SGD(model.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
    
    loss_per_epoch_list = []
    acc_per_epoch_list = []
    
    for epoch in range(epochs):
        loss_list = []    
            
        for x, labels in trainloader:
            x, labels = x.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            loss = torch.nn.CrossEntropyLoss()(outputs, labels.long())
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
    
        loss_per_epoch_list.append(np.mean(loss_list))
        acc_per_epoch_list.append(test(model, validloader, device))
        scheduler.step()
        
        clear_output(wait=False)
        plt.plot(loss_per_epoch_list)
        plt.savefig('graphs/loss.png')
        plt.show()
        plt.plot(acc_per_epoch_list)
        plt.savefig('graphs/acc.png')
        plt.show()
    return test(model, testloader, device)

# Training model

In [6]:
train_and_validate(model, trainloader, validloader, testloader, epochs, device = device)

# Test model

In [6]:
acc, full_labels, full_outputs = test(model, testloader, device)

print(acc)

0.92


# Save model

In [7]:
torch.save(model.state_dict(), 'resnet18_final.pt')